<a href="https://colab.research.google.com/github/LukasB10/CACI_SWP/blob/master/Two_model_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [0]:
data_raw = pd.read_csv("fashionB_clean.csv")

In [33]:
data_raw.head(10)

,checkoutAmount,converted,InitCartNonEmpty,VisitorKnown,ViewsOnsearch,ViewsOnproduct,ViewsOnoverview,ViewsOnsale,InitPageWasproduct,InitPageWashome,InitPageWassale,ViewedBeforecart,HadCartAdd,WasConvertedBefore,PreviousVisitCount,TimeSpentOnsearch,TimeSpentOnproduct,TimeSpentOnoverview,TimeSpentOnsale,SessionTimeInSeconds,SecondsFor3,ClicksPerproduct,ChannelIsEMAIL,ChannelIsPAID,ChannelIsSEARCH,NumberOfDifferentoverview,NumberOfDifferentproduct,CartQuantity,NormalizedCartSum,ViewCountLastVisit,ScreenWidth,DurationLastVisitInSeconds,VisitCountLastWeek,VisitCountToday,PageIshome,PageIssearch,PageIsproduct,PageIsoverview,PageIssale,RepeatCount,SameAsPreviousPrevious,TriggerEventsSinceLastOnThisScreenType,TriggerEventsSinceLastOnThisPage,MonetaryValueOfPreviousSessions,MonetaryDiscountValueOfPreviousSessions,ClicksOnLastScreenCount,TotalClickCount,HasConfirmedBefore,HasAbortedBefore,IsMobile,HourOfDay3hBlock1,HourOfDay3hBlock2,HourOfDay3hBlock3,HourOfDay3hBlock4,HourOfDay3hBlock5,HourOfDay3hBlock6,HourOfDay3hBlock7,HourOfDay3hBlock8,DayOfWeek2,DayOfWeek3,DayOfWeek4,DayOfWeek5,DayOfWeek6,DayOfWeek7,TREATMENT,TREATMENT_EFFECT_CONVERSION,TREATMENT_EFFECT_RESPONSE
0,0.000000,0,0,1,0,3,1,0,0,1,0,0,0,0,1,0,25,19,0,53,25,5,0,0,1,1,3,0,0.000000,3,1366,34,1,0,0,0,1,0,0,0,0,1,0,0.00,0.0,0,7,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0.017744,1.913716
1,0.000000,0,0,1,0,0,0,4,0,1,0,0,0,0,4,0,0,0,65,82,54,0,1,0,0,0,0,1,50.189333,3,1280,1069,0,0,0,0,0,0,1,1,0,1,1,0.00,0.0,1,4,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0.018124,1.358194
2,0.000000,0,0,1,0,3,1,0,0,1,0,0,0,0,19,0,22,28,0,53,22,0,0,0,0,1,3,0,0.000000,4,1366,79,2,0,0,0,1,0,0,0,0,1,0,0.00,0.0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0.016405,1.760658
3,0.000000,0,0,1,0,0,0,4,0,1,0,0,1,1,5,0,0,0,129,134,57,0,0,0,0,0,0,1,31.385147,35,1536,514,0,0,0,0,0,0,1,3,1,1,1,1375.36,0.0,0,4,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0.034350,2.008954
4,0.000000,0,0,1,0,1,0,3,0,1,0,0,0,0,14,0,24,0,21,53,33,2,0,0,0,0,1,0,0.000000,57,1200,1249,0,0,0,0,0,0,1,1,1,2,2,0.00,0.0,0,5,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0.007788,0.529630
5,0.000000,0,0,0,0,3,0,2,0,0,1,0,0,0,0,0,9,0,60,69,9,1,1,0,0,0,2,0,0.000000,0,1680,0,0,0,0,0,1,0,0,1,1,1,2,0.00,0.0,0,3,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0.085121,8.665788
6,81.801659,1,0,0,0,1,3,0,0,1,0,0,0,0,0,0,291,88,0,405,88,5,0,0,1,1,1,0,0.000000,0,1344,0,0,0,0,0,0,1,0,2,1,1,1,0.00,0.0,0,8,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0.178082,14.567372
7,0.000000,0,0,0,0,1,3,0,0,1,0,0,0,0,0,0,14,8,0,41,19,1,0,0,1,1,1,1,34.066335,0,1600,0,0,0,0,0,0,1,0,2,1,2,2,0.00,0.0,0,4,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0.065692,5.919806
8,0.000000,0,0,1,0,4,0,1,1,0,0,0,0,0,6,0,2028,0,7,2035,12,2,0,1,0,0,3,0,0.000000,11,1536,188,2,0,0,0,1,0,0,1,1,1,2,0.00,0.0,0,2,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0.118116,8.308859
9,0.000000,0,0,1,0,1,0,4,0,0,1,0,0,0,2,0,13,0,121,134,31,0,1,0,0,0,1,0,0.000000,45,1324,2512,1,0,0,0,0,0,1,0,0,1,0,0.00,0.0,0,3,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,-0.018449,-1.331191


In [0]:
data = data_raw.drop(['converted', 'TREATMENT_EFFECT_CONVERSION', 'TREATMENT_EFFECT_RESPONSE'], axis = 1)

In [0]:
treat = data.loc[data.TREATMENT == 1, ]
control = data.loc[data.TREATMENT == 0, ]

In [0]:
X_treat = treat.drop(['checkoutAmount'], axis = 1)
y_treat = treat[['checkoutAmount']]

X_control = control.drop(['checkoutAmount'], axis = 1)
y_control = control[['checkoutAmount']]

In [0]:
# Split into train and test data
from sklearn.model_selection import train_test_split

seed = 123

X_treat_train, X_treat_test, y_treat_train, y_treat_test = train_test_split(X_treat, y_treat, test_size = 0.30, random_state = seed)
X_control_train, X_control_test, y_control_train, y_control_test = train_test_split(X_control, y_control, test_size = 0.30, random_state = seed)

In [0]:
# Standardization
from sklearn.preprocessing import MinMaxScaler

# Fit Scaler
scaler_X_treat = MinMaxScaler().fit(X_treat_train) # only X_train values are used not to leak the data to the test set
scaler_y_treat = MinMaxScaler().fit(y_treat_train)
scaler_X_control = MinMaxScaler().fit(X_control_train)
scaler_y_control = MinMaxScaler().fit(y_control_train)

X_treat_train_standardized = scaler_X_treat.transform(X_treat_train)
X_treat_test_standardized = scaler_X_treat.transform(X_treat_test)

y_treat_train_standardized = scaler_y_treat.transform(y_treat_train)
y_treat_test_standardized = scaler_y_treat.transform(y_treat_test) 

X_control_train_standardized = scaler_X_control.transform(X_control_train)
X_control_test_standardized = scaler_X_control.transform(X_control_test) 

y_control_train_standardized = scaler_y_control.transform(y_control_train)
y_control_test_standardized = scaler_y_control.transform(y_control_test)  

In [66]:
X_treat_test_tensor.dtype

torch.float32

In [0]:
X_treat_train_tensor = torch.from_numpy(X_treat_train_standardized.astype(np.float32))
X_treat_test_tensor = torch.from_numpy(X_treat_test_standardized.astype(np.float32))

y_treat_train_tensor = torch.from_numpy(y_treat_train_standardized.astype(np.float32))
y_treat_test_tensor = torch.from_numpy(y_treat_test_standardized.astype(np.float32))

X_control_train_tensor = torch.from_numpy(X_control_train_standardized.astype(np.float32))
X_control_test_tensor = torch.from_numpy(X_control_test_standardized.astype(np.float32))

y_control_train_tensor = torch.from_numpy(y_control_train_standardized.astype(np.float32))
y_control_test_tensor = torch.from_numpy(y_control_test_standardized.astype(np.float32))

In [0]:
import torch
import torch.nn as nn

In [15]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [0]:
class Feedforward(torch.nn.Module):
        def __init__(self, input_size, hidden_size):
            super(Feedforward, self).__init__()
            self.input_size = input_size
            self.hidden_size  = hidden_size
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.relu = torch.nn.ReLU()
            self.fc2 = torch.nn.Linear(self.hidden_size, 1)
            self.sigmoid = torch.nn.Sigmoid()
            
        def forward(self, x):
            hidden = self.fc1(x)
            relu = self.relu(hidden)
            output = self.fc2(relu)
            output = self.sigmoid(output)
            return output

In [0]:
model = Feedforward(63, 16)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [68]:
model.eval()
y_pred = model(X_treat_test_tensor)
before_train = criterion(y_pred.squeeze(), y_treat_test_tensor)
print('Test loss before training' , before_train.item())

Test loss before training 0.949738621711731


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([26722, 1])) that is different to the input size (torch.Size([26722])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
